# Rank aggregation with Borda Fusion

## Some background
A rank aggregation problem involves merging some number of preference lists to produce a single new list containing the 'group' preferences. The algorithm I have chosen to use (at least, for now, as a first attempt) is known as borda fusion or borda count. The principle is simple: convert the "rank" into points and sum them up across users. Then sort the list based on rank.

In borda count, more points are assigned to entries with a higher rank - i.e. the first ranked item gets the most points. If there are $k$ items to be ranked, then an item ranked $n$ gets $(k-n)$ points.

---

## implementing this in python

Implementing this idea in python isnt really all that hard - but there is some nuance, especially with how the data is modelled. First, lets load in some data:

In [1]:
import pandas as pd
import numpy as np

In [2]:
# load environment variables
%load_ext dotenv
%dotenv
    
# grab the one we need
import os
POSTGRES_URL = os.environ.get("POSTGRES_URL_PYTHON")

In [3]:
from src.apy.managers.sqlManager import SqlManager

sql_repo = SqlManager(POSTGRES_URL)

data = sql_repo.get_club_ranks("tea-club")
data

,user,item,preference
0,user_2Wl5UZ7BeqxPZWtHgwMy5th9Qnj,tt0080684,6.0
1,user_2Wl5UZ7BeqxPZWtHgwMy5th9Qnj,tt0076759,5.0
2,user_2Wl5UZ7BeqxPZWtHgwMy5th9Qnj,tt0086190,4.0
3,user_2Wl5UZ7BeqxPZWtHgwMy5th9Qnj,tt0167261,3.0
4,user_2Wl5UZ7BeqxPZWtHgwMy5th9Qnj,tt0167260,2.0
5,user_2Wl5UZ7BeqxPZWtHgwMy5th9Qnj,tt0120737,1.0


Here I am using a test club with some faked users, just so we have some proper data to work with. The data reflect the nature of our data model and DB tables; This is a single list, with the preference of all different users globbed together. One row of the table looks like This
```python
{
    "user": movie.userID,
    "item": movie.movieID,
    "preference": movie.preference,
}
```
This makes the next step easy - we need a list of all the unique movie IDs in this combined set; We can do this by selecting out the movieID column and grabbing the unique movies.


In [4]:
movies = data["item"].unique()
k = len(movies)

Now we are ready to do the actual borda fusion; This can be done very easily with pandas. First, we need to break the input data into groups based on user;
for each of these smaller tables, we need to compute a new column - `borda-points`. Recall: this should be our number of total movies $k$ minus the rank of the movie. How fortunate that pandas has a `df.rank()` function built in!

We then need to join these per-user datafranes back together - which we can do with `pd.concat()`.

We dont need columns other than `["item", "borda-points"]` so lets just grab those. Finally, we can group the entries by item, and sum the borda points, and then sort by the resulting totals

In [5]:
def add_points(l):
    l["borda-points"] = k - l["preference"].rank(ascending=True)
    return l

result = pd.concat([add_points(rankings) for _, rankings in data.groupby("user")])[["item", "borda-points"]].groupby("item").sum().sort_values(by="borda-points")
result

,borda-points
item,
tt0080684,0.0
tt0076759,1.0
tt0086190,2.0
tt0167261,3.0
tt0167260,4.0
tt0120737,5.0
